In [2]:
import csv, re
import pandas as pd

In [35]:
vcffile = '../tests/vcfs/test.vcf'
#vcffile = '../tests/vcfs/010422/Oncomine_Variants_(5.16)_filtered.vcf'

def parse_thermo_vcf(vcf):
    ''' Les inn vcf til pandas dataframe'''
    df = pd.read_csv(vcffile, sep="\t", comment='#', names=["CHROM","POS","ID","REF","ALT","QUAL","FILTER","INFO","FORMAT","GT"])
    return df
   
def filter_nocalls(df):
    ''' Fjern varianter som begynner med 0/0 eller ./.'''
    dfOut = df[   (~(df['GT'].str.startswith('./.')))   & (~(df['GT'].str.startswith('0/0')))   ]
    return dfOut

def explode_format_gt(df):
    ''' Eksploderer GT OG FORMAT-kolonnene til egne kolonner
    Følgende kolonner finnes to steder, så de fjernes fra format: AF,AO,DP,FAO,FDP,FRO,FSAF,FSAR,FSRF,FSRR,RO,SAF,SAR,SRF,SRR'''
    df.reset_index(inplace=True,drop=True)
    ny = pd.DataFrame(list(dict(zip(a,b)) for a,b in zip(df['FORMAT'].str.split(":"), df['GT'].str.split(":"))))
    for i in ["AF","AO","DP","FAO","FDP","FRO","FSAF","FSAR","FSRF","FSRR","RO","SAF","SAR","SRF","SRR"]:
        try:
            ny = ny.drop(i, axis=1)
        except:
            pass
    dfOut = pd.concat([df, ny], axis=1)
    del dfOut["GT"]
    del dfOut["FORMAT"]
    return dfOut

def replace_semi(inputstring):
    '''  Hvor det finnes semikolon inne i curly braces i INFO-kolonnen gjoeres denne om til underscore - ellers blir det feilmelding'''
    return re.sub(r"{[^{}]+}", lambda x: x.group(0).replace(";", "_"), inputstring)

def explode_info(df):
    ''' Eksploderer INFO-kolonnen til egne kolonner'''
    df.reset_index(inplace=True,drop=True)
    # First maa vi fjerne alle semikolon som er inne i klammer:
    df["INFO"] = df["INFO"].apply(replace_semi)
    # Replace HS MED HS=NA
    df['INFO'] = df['INFO'].str.replace(';HS;',';HS=NA;')
    ny2 = pd.DataFrame([dict(w.split('=', 1) for w in x) for x in df["INFO"].str.split(";")])
    dfOut = pd.concat([df, ny2], axis=1)
    del dfOut["INFO"]
    return dfOut

In [36]:
df = parse_thermo_vcf(vcffile)
df = filter_nocalls(df)
df = explode_format_gt(df)
df = explode_info(df)

In [37]:

l = []
for i in df.columns:
    if i in l:
        print i
    l.append(i)


AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [102]:
[dict(i.split('=') for i in x) for x in list(mini['INFO'].str.split(";"))]


ValueError: dictionary update sequence element #44 has length 3; 2 is required

In [78]:
d = [dict(x.split('=') for i in x) for x in mini['INFO'].str.split(';')]
d
#{k: v for d in L for k, v in d.items()}
#df = pd.DataFrame.from_dict()
#pd.DataFrame.from_dict(L)
#df = df[['name']].join(pd.DataFrame(L))

AttributeError: 'list' object has no attribute 'split'

In [57]:
{k: [v] for k, v in L.items()}

AttributeError: 'list' object has no attribute 'items'

In [11]:
[dict(i.split(';') for i in x) for x in mini['INFO']]

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [ ]:
def expand_info_col(df):